# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [32]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

Peer closed connection.


In [30]:
contract = contracts[0]
ib.qualifyContracts(contract)
ib.reqMktData(contract, '', False, False)
ticker = ib.ticker(contract)
ticker.ask, ticker.bid

(1.08774, 1.08773)

### Streaming tick data

Create some Forex contracts:

In [25]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

Request streaming ticks for them:

In [28]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

ticker = ib.ticker(eurusd)

Wait a few seconds for the tickers to get filled.

The price of Forex ticks is always nan. To get a midpoint price use ``midpoint()`` or ``marketPrice()``.

The tickers are kept live updated, try this a few times to see if the price changes:

In [29]:
ticker.bid

1.0877

The following cell will start a 30 second loop that prints a live updated ticker table.
It is updated on every ticker change.

In [8]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(30)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,4500000.0,1.08652,1.08653,2000000.0,1.0916,1.0859,1.0904
USDJPY,1000000.0,154.877,154.881,1000000.0,156.49,154.7,156.07
GBPUSD,1000000.0,1.27631,1.27633,400000.0,1.28175,1.2743,1.2808
USDCHF,1500000.0,0.89306,0.8931,3000000.0,0.89725,0.89225,0.89575
USDCAD,2000000.0,1.36772,1.36777,3500000.0,1.3699,1.36205,1.3627
AUDUSD,2000000.0,0.66426,0.66428,5500000.0,0.6699,0.6631,0.66895


New tick data is available in the 'ticks' attribute of the pending tickers.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [9]:
for contract in contracts:
    ib.cancelMktData(contract)

### Tick by Tick data ###

The ticks in the previous section are time-sampled by IB in order to cut on bandwidth. So with ``reqMktdData`` not every tick from the exchanges is sent. The promise of ``reqTickByTickData`` is to send every tick, just how it appears in the TWS Time & Sales window. This functionality is severly nerfed by a total of just three simultaneous subscriptions, where bid-ask ticks and sale ticks also use up a subscription each.

The tick-by-tick updates are available from ``ticker.tickByTicks`` and are signalled by ``ib.pendingTickersEvent`` or ``ticker.updateEvent``.

In [10]:
ticker = ib.reqTickByTickData(eurusd, 'BidAsk')
ib.sleep(2)
print(ticker)

ib.cancelTickByTickData(ticker.contract, 'BidAsk')

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2024, 6, 4, 12, 46, 1, 850932, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.08668, bidSize=11500000.0, ask=1.08671, askSize=7500000.0, prevBid=1.08662, prevBidSize=11000000.0, prevAsk=1.0867, prevAskSize=6500000.0, high=1.0916, low=1.0859, close=1.0904, halted=0.0, tickByTicks=[TickByTickBidAsk(time=datetime.datetime(2024, 6, 4, 12, 46, 1, 850932, tzinfo=datetime.timezone.utc), bidPrice=1.08668, askPrice=1.08671, bidSize=11500000.0, askSize=7500000.0, tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False))])


### Historical tick data

Historical tick data can be fetched with a maximum of 1000 ticks at a time. Either the start time or the end time must be given, and one of them must remain empty:

In [13]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks

[HistoricalTickBidAsk(time=datetime.datetime(2024, 6, 4, 12, 45, 36, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08667, priceAsk=1.08669, sizeBid=8500000.0, sizeAsk=5000000.0),
 HistoricalTickBidAsk(time=datetime.datetime(2024, 6, 4, 12, 45, 37, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08667, priceAsk=1.08669, sizeBid=8500000.0, sizeAsk=5500000.0),
 HistoricalTickBidAsk(time=datetime.datetime(2024, 6, 4, 12, 45, 37, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08667, priceAsk=1.08669, sizeBid=8500000.0, sizeAsk=4500000.0),
 HistoricalTickBidAsk(time=datetime.datetime(2024, 6, 4, 12, 45, 37, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.08667, priceAsk=1.08669, sizeBid=7500000.0, sizeAsk=5000000.0),
 HistoricalT

In [31]:
ib.disconnect()